In [ ]:
# 📌 Mount Google Drive to access the dataset
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')
#Load the walmart sales Dataset from drive
wal_data="/content/drive/MyDrive/ml_engineer/p4_Consumer_demand_Walmart/walmart.csv"
df=pd.read_csv(wal_data)
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,1422
3,1000001,P00085442,F,0-17,10,A,2,0,12,1057
4,1000002,P00285442,M,55+,16,C,4+,0,8,7969


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# ✅ Cargar tu dataset aquí si no lo has hecho
# df = pd.read_csv('ruta/del/archivo.csv')

# ✅ Tomar una muestra del 30%
df_sample = df.sample(frac=0.3, random_state=42)

# 🧼 Limpiar Stay_In_Current_City_Years
df_sample['Stay_In_Current_City_Years'] = df_sample['Stay_In_Current_City_Years'].replace('4+', 4).astype(int)

# 🧠 Codificar variables categóricas básicas (sin Product_ID todavía)
df_encoded = pd.get_dummies(df_sample[['Gender', 'Age', 'City_Category']], drop_first=True)

# 🔗 Combinar features numéricos
X = pd.concat([
    df_encoded,
    df_sample[['Occupation', 'Stay_In_Current_City_Years', 'Marital_Status', 'Product_Category']]
], axis=1)

y = df_sample['Purchase']
# Dividir train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modelo Random Forest (versión liviana)
model = RandomForestRegressor(n_estimators=50, random_state=42)  # Menos árboles
model.fit(X_train, y_train)

# Predecir
y_pred = model.predict(X_test)

# Evaluación
print(f'RMSE: {np.sqrt(mean_squared_error(y_test, y_pred)):.2f}')
print(f'R² Score: {r2_score(y_test, y_pred):.4f}')



RMSE: 3032.50
R² Score: 0.6354


Although a predictive model was built to estimate average purchase amounts for inventory optimization, the model yielded an RMSE of ~$3,000. Given that this represents a high error margin relative to the target range, the predictions were not considered production-ready. Further feature engineering and modeling (e.g., XGBoost, user segmentation, or log transformations) are proposed for future iterations.


In [ ]:
inventory_estimates.sort_values('Estimated_Avg_Purchase', ascending=False).head(10)


,Product_Category,City_Category,Estimated_Avg_Purchase
29,10,C,19486.0
27,10,A,19243.0
28,10,B,19235.0
20,7,C,17198.0
26,9,C,16314.0
17,6,C,16181.0
18,7,A,15950.0
19,7,B,15797.0
16,6,B,15747.0
15,6,A,15576.0
